# Numerical Optimization using JuMP


# JuMP

[JuMP](https://github.com/JuliaOpt/JuMP.jl) is a Julia package that provides a modeling language for general optimization problems.

## Optimization in Julia

Julia's optimization packages have become popular due to their ease of use, and variety of interfaces to mature solvers.  The main optimization functionality in Julia is provided by [JuliaOpt](http://www.juliaopt.org/).

The two high-level interfaces that you are most likely to use are
* [JuMP](https://github.com/JuliaOpt/JuMP.jl) - a modeling language for all sorts of optimization problems
* [Convex.jl](https://github.com/JuliaOpt/Convex.jl) - a package for disciplined convex programming (like [CVX](http://cvxr.com/cvx/))

There is a mid-level interface as well, [MathProgBase.jl](https://github.com/JuliaOpt/MathProgBase.jl).

The powerful thing about all of the above is that it is largely **solver independent**. This means you can forumlate the optimization problem with these packages, and then choose from a variety of solvers to use under the hood.  This is just like other modeling languages like AMPL - the reason why Julia's optimization packages have become popular is that they are generally easier to use than older modeling languages.

## Solvers

Today is more about turning your optimization models into something that can run on a computer via JuMP, but you still need a solver to actually solve the problem for you under the hood.  [JuliaOpt](http://www.juliaopt.org/) has a list of solvers that can be called from the high level interfaces (there are currently 20).  There are many open-source options available, but there are also interfaces to some of the big commercial solvers such as [Gurobi](http://www.gurobi.com/), [Mosek](https://www.mosek.com/), [Knitro](https://www.artelys.com/en/optimization-tools/knitro), etc. Many of these commercial solvers offer free academic/student licences, and if you are trying to solve large optimization problems they may be worth looking at.

In [1]:
# If you have not already installed JuMP and a solver
Pkg.add("JuMP")
Pkg.add("Clp")   # solver
Pkg.add("Ipopt") # solver

INFO: Package JuMP is already installed
INFO: METADATA is out-of-date — you may not have the latest version of JuMP
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Package Clp is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Clp
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [33]:
using JuMP
using Clp
using Cbc

# Optimization Problems

The hardest part of solving an optimization problem is often formulating the problem itself.  Optimization problems generally take the form

$$
\text{minimize}_x~f(x)\\
\text{subject to}~c_e(x) = 0; c_i(x) \ge 0
$$

where $f(x)$ is some function, $c_e(x)$ encodes equality constraints, and $c_i(x)$ encodes inequality constraints.

# Linear Programming

[Linear Programs (LPs)](https://en.wikipedia.org/wiki/Linear_programming) are optimization problems for which both the objective and constraint functions are linear.

$$
\text{minimize}_{x\in \mathbb{R}^n}~c^T x\\
\text{subject to}~Ax = a; Bx \ge b
$$

One place LPs arise are in maximizing profit.

Here's a very simple LP we can easily solve by hand:

$$
\text{minimize}~x+y\\
\text{subject to:}\\
\qquad x\ge 0\\
\qquad y \ge 0\\
\qquad x+y \le 1
$$

In [26]:
using Cbc

In [22]:
# create a model
# m = Model(solver=ClpSolver())
m = Model(solver=GurobiSolver())
# add variables
@variable(m, x >= 0)
@variable(m, y >= 0)
# add additional constraint
@constraint(m, x+y <= 1)
# define objective
@objective(m, Min, x+y)
# prints problem we've defined
m

Minimization problem with:
 * 1 linear constraint
 * 2 variables
Solver is Gurobi

In [23]:
# solve the problem
solve(m)
# print objective value, and the values of x,y
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

Academic license - for non-commercial use only
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00
Optimal objective: 0.0. x = 0.0 y = 0.0


# Exercise

1. Install another solver that will solve a Linear Program (see [JuliaOpt](http://www.juliaopt.org/) for options - look at the first column), and solve the above example using the new solver.

# Quadratic Programming

[Quadratic Programs (QPs)](https://en.wikipedia.org/wiki/Quadratic_programming) allow the objective function $f(x)$ to be quadratic.  The optimization problems have the form

$$
\text{minimize}_{x\in \mathbb{R}^n}~\tfrac{1}{2}x^T Q x+c^T x\\
\text{subject to}~Ax \le b
$$

Note that in the case that $Q$ is SPD, that the problem is convex, but this is generally not the case.

In [29]:
using Ipopt

INFO: Precompiling module Ipopt.


In [33]:
# create a model
m = Model(solver=IpoptSolver())
# add variables
@variable(m, x >= 0)
@variable(m, y >= 0)
# add additional constraint
@constraint(m, x+y <= 1)
# define objective
@objective(m, Min, (x-0.5)^2 + y)
# prints problem we've defined
m

Minimization problem with:
 * 1 linear constraint
 * 2 variables
Solver is Ipopt

In [34]:
solve(m)
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

# Second-order Cone Constraints

http://www.juliaopt.org/JuMP.jl/0.18/refmodel.html#second-order-cone-constraints

# Integer Programming

[Integer Programs (IPs)](https://en.wikipedia.org/wiki/Integer_programming) add integer constraints to the optimization variables.  This makes sense when variables come in unit quantities (e.g. people).

In [48]:
using JuMP, Cbc
m = Model(solver=CbcSolver())
@variable(m, x >= 0, Int) # Int keyword says that x should be an integer
@variable(m, y >= 0, Int)

@objective(m, Max, x + y)
@constraint(m, 50x + 24y <= 2400)
@constraint(m, 30x + 33y <= 2100)
m

Maximization problem with:
 * 2 linear constraints
 * 2 variables: 2 integer
Solver is CbcMathProg

In [49]:
solve(m)
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

Optimal objective: 66.0. x = 31.0 y = 35.0


# Mixed Integer Programming

Mixed Integer Programs (MIPs) have some variables that are constrained to be integers, and some that are not.

callbacks:

http://www.juliaopt.org/JuMP.jl/0.18/callbacks.html

In [2]:
using JuMP, Cbc
m = Model(solver=CbcSolver())
@variable(m, x >= 0, Int) # Int keyword says that x should be an integer
@variable(m, y >= 0)

@objective(m, Max, x + y)
@constraint(m, 50x + 24y <= 2400)
@constraint(m, 30x + 33y <= 2100)
m

Maximization problem with:
 * 2 linear constraints
 * 2 variables: 1 integer
Solver is CbcMathProg

In [3]:
solve(m)
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

Optimal objective: 66.41666666666666. x = 31.0 y = 35.41666666666667


# Exercise - NP-complete problems

[NP-complete](https://en.wikipedia.org/wiki/NP-completeness) problems are decision problems that may have no polynomial-time algorithm to compute, although solutions can be verified in polynomial time.  They arise in computer science, operations research, and have a variety of applications.

Many of these problems can be formulated as optimization problems with integer constraints.

* Minimum Vertex Cover https://en.wikipedia.org/wiki/Vertex_cover
* Maximum Coverage https://en.wikipedia.org/wiki/Maximum_coverage_problem
* Max-Cut (Need commercial solver for QIP)


## Maximum-Coverage

The Maximum-Coverage problem is as follows: given $m$ sets, and an integer $k$, maximize the number of elements covered by at most $k$ sets. We'll assume that each of the $m$ sets is a subset of $N$ elements.

An application to think of: you want to buy $N$ items.  There are $m$ stores you could possibly go to, and each store only carries some of the items you'd like to buy.  Unfortunately you only have time to go to $k$ of the stores, but you'd like to get as much of your shopping done as possible (maximize the number of items you get).  Which stores should you go to today?

### Step 1: Formulate an Integer Program

In [1]:
# generates a Maximum-Coverage problem
N = 15
nS = 6
k = 2
x = collect(1:N) # set of items
# create assign elements to the nS sets
S = Vector{Vector{Int64}}(N)
for i = 1:N
   S[i] = unique([rand(1:nS) rand(1:nS)])
end
;

In [18]:
using JuMP, Cbc
m = Model(solver=CbcSolver())

@variable(m, 0 <= y[1:N] <= 1, Int) # denotes if element in chosen set
@variable(m, 0 <= x[1:nS] <= 1, Int) # denotes if set is chosen
@objective(m, Max, sum(y))

@constraint(m, sum(x) <= k) # choose at most k sets

# if y[i] is chosen, then it must be in at least 1 chosen set
for i = 1:N
    @constraint(m, sum(x[S[i]]) >= y[i])
end

m

Maximization problem with:
 * 16 linear constraints
 * 21 variables: 21 integer
Solver is CbcMathProg

In [19]:
solve(m)
@show getvalue(x)
sum(getvalue(y))

getvalue(x) = [0.0, 1.0, 1.0, 0.0, 0.0, 0.0]


12.0

In [17]:
# random assignment
# choose nS random sets
x_rand = zeros(nS)
for i = 1:k
    found = false
   while !found
       i = rand(1:nS)
        if x_rand[i] == 0
            x_rand[i] = 1
            found = true
        end
    end
end
@show x_rand
y_rand = zeros(N)
for i = 1:N
   if sum(x_rand[S[i]]) > 0
        y_rand[i] = 1
    end
end
sum(y_rand)

x_rand = [1.0, 0.0, 1.0, 0.0, 0.0, 0.0]


10.0

### Step 2: LP relaxation

We can turn the integer program to a linear program by relaxing the constraint that the variables need to be integers.  After solving the LP, we then round the solution to integers 

In [29]:
using JuMP, Clp
m = Model(solver=ClpSolver())

@variable(m, 0 <= y[1:N] <= 1) # denotes if element in chosen set
@variable(m, 1>= x[1:nS] >= 0) # denotes if set is chosen
@objective(m, Max, sum(y))

@constraint(m, sum(x) <= k) # choose at most k sets

# if y[i] is chosen, then it must be in at least 1 chosen set
for i = 1:N
    @constraint(m, sum(x[S[i]]) >= y[i])
end

m

Maximization problem with:
 * 16 linear constraints
 * 21 variables
Solver is ClpMathProg

In [30]:
solve(m)
@show getvalue(x)
sum(getvalue(y))

getvalue(x) = [0.0, 1.0, 1.0, 0.0, 0.0, 0.0]


12.0

# Exercise - Max-Cut of a Graph

The [Max-Cut](https://en.wikipedia.org/wiki/Maximum_cut) problem is to partition the vertices $V$ of a weighted graph $G = (V,E,W)$ into two sets $S,\bar{S}$ that maximizes the weight of the cut edges.
$$ 
\text{maximize}_S ~\sum_{i\in S} \sum_{j\in\bar{S}} w_{ij}
$$
We see that this is equivalent to the integer QP (http://www-math.mit.edu/~goemans/PAPERS/maxcut-jacm.pdf)
$$
\text{maximize} ~\tfrac{1}{2} \sum_{i<j} w_{ij} (1-y_i y_j)\\
\text{subject to:} ~ y_i\in\{-1,1\}
$$
We see that maximizing the above objective is equivalent to maximizing
$$
-\sum_{i<j} w_{ij} y_i y_j = y^T W y
$$
Where $W$ is the weighted adjacency matrix of $G$


To simplify everything, we'll assume all weights are 1, so that $W$ is just the adjacency matrix of the graph.  We'll also work on a graph that we know the answer to.


In [59]:
# line graph
A = 1.0*[0 1 0; 1 0 1; 0 1 0]
A

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 1.0  0.0  1.0
 0.0  1.0  0.0

In [94]:
function obj_value(y, W)
   return 0.25*(sum(W) -y'*W*y)
end

# if weights are 0, randomly assign
function round_weights{T}(y::Vector{T})
    n = length(y)
    z = Array{T}(n)
    @simd for i = 1:n
        z[i] = (y[i] == T(0)) ? sign(rand()) : sign(y[i])
    end
    return z
end

# random weights
y = round_weights(randn(3))
obj_value(y, A)

1.0

## Part 1 - Relaxation

We can relax the above problem to:
$$
\text{maximize}_y ~-y^T W y\\
\text{subject to:} ~-1 \le y_i \le y
$$
And round the solution to $1$ or $-1$

1. What sort of optimization problem is the relaxation? (LP, QP, IP, MIP, ...)  What's an appropriate solver to use?
2. Use JuMP to model the above problem, find the solution, and round the vertex weights.
3. Try both graphs.

In [70]:
using Ipopt
m = Model(solver=IpoptSolver())
@variable(m, -1 <= y[1:3] <= 1) # Int keyword says that x should be an integer
@objective(m, Max, -y'*A*y)
m

Maximization problem with:
 * 0 linear constraints
 * 3 variables
Solver is Ipopt

In [71]:
solve(m)

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

:Optimal

In [72]:
getvalue(y)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

## Part 2 - Full Integer Program

1. use JuMP to model the original integer program. What is an appropriate solver to use?
2. Which option is faster?

In [85]:
using SCIP
# m = Model(solver=AmplNLSolver("couenne"))
m = Model(solver=SCIPSolver())
@variable(m, -1 <= y[1:3] <= 1, Int) 
@objective(m, Max, -y'*A*y)
m

INFO: Precompiling module SCIP.


Maximization problem with:
 * 0 linear constraints
 * 3 variables: 3 integer
Solver is SCIP

In [86]:
solve(m)
yrnd = round_weights(getvalue(y))
obj_value(yrnd, A)

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 3, fast)       0 del vars, 0 del conss, 0 add conss, 2 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 2 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 4 variables (0 bin, 3 int, 0 impl, 1 cont) and 1 constraints
      1 constraints of type <quadratic>
Presolving Time: 0.00
transformed 1/3 original solutions to the transformed problem space
Quadratic constraint handler does not have LAPACK for eigenvalue comput

2.0

In [87]:
yrnd

3-element Array{Float64,1}:
  1.0
 -1.0
  1.0

# Non-linear Programming

JuMP also allows you to model more general [nonlinear problems (NLPs)](https://en.wikipedia.org/wiki/Nonlinear_programming).  To do this, you will want to use the macros `@NLobjective`, and `@NLconstraint`.  These can be combined with objectives and constraints we've already seen.

* JuMP's Introduction to solving NLPs [here](http://www.juliaopt.org/JuMP.jl/0.18/nlp.html)

## Example

Here, we'll look at the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function)
$$
f(x,y) = (a-x)^2 + b(y-x^2)^2
$$
For $b>0$, this function has a global minimum at $(x,y) = (a,a^2)$, where $f(x,y) = 0$.

In [5]:
using Plots
plotlyjs()
a = 1.0
b = 50.0
f(x,y) = (a - x)^2 + b*(y - x^2)^2
n = 100
xs = ones(n)*linspace(-1.5,1.5,n)'
ys = linspace(-1.5,1.5,n)*ones(n)'
fs = f.(xs, ys)
surface(xs, ys, fs)

In [11]:
# From http://www.juliaopt.org/JuMP.jl/0.18/nlp.html
using JuMP, Ipopt
m = Model(solver=IpoptSolver())
@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)

solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

## Automatic Differentiation

Most optimization solvers need gradients and hessians in order to work.  How does JuMP obtain this information?

The answer is that all this information is obtained through [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) which uses the rules of differential calculus to differentiate functions just like you would.  Note that **this is different from using finite difference schemes** and is typically much more accurate.

The packages that JuMP uses for automatic differentiation are
* [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl)
* [Calculus.jl](https://github.com/JuliaMath/Calculus.jl)

You can also use these packages for your own purposes outside of JuMP.  As long as a function is built from core functions (e.g. you aren't calling special function libraries), products and sums, you can differentiate it.

In [12]:
using ForwardDiff

f(x::Vector) = sum(x)
n = 5
x = randn(5)
f(x)

0.3691272189343932

In [13]:
ForwardDiff.gradient(f,x)

5-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0

In [14]:
ForwardDiff.hessian(f,x)

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

You can also wrap the gradient and hessian functions

In [20]:
g(x) = ForwardDiff.gradient(f,x)
g(x)

5-element Array{Float64,1}:
  0.924226 
 -0.0349832
  0.0      
  0.0      
  0.0      

In [22]:
# an example on a more complicated function
f(x::Vector) = x[1]*x[2] + sum(sin.(x[3:end]))
g(x) = ForwardDiff.gradient(f,x)
g(x)

5-element Array{Float64,1}:
  0.924226 
 -0.0349832
  0.746722 
  0.834778 
  0.930322 

In [26]:
@time f(x)

  0.000007 seconds (7 allocations: 400 bytes)
  0.000090 seconds (10 allocations: 1.328 KiB)


5-element Array{Float64,1}:
  0.924226 
 -0.0349832
  0.746722 
  0.834778 
  0.930322 

## Using Custom Functions for NLP in JuMP

Above, we wrote the Rosenbrock function explicitly as the objective function
```julia
@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)
```
We can also provide a function wrapper for the function - all we need to do is "register" the function in JuMP.  You can find informaiton in [JuMP's documentation here](http://www.juliaopt.org/JuMP.jl/0.18/nlp.html#user-defined-functions).

In [28]:
using JuMP, Ipopt

rosenbrock(x,y) = (a - x)^2 + b*(y - x^2)^2

m = Model(solver=IpoptSolver())

# registers function with JuMP, and derivatives are computed
JuMP.register(m, :rosenbrock, 2, rosenbrock, autodiff=true)

@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, rosenbrock(x,y))

solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

Let's break down the register command:
```julia
JuMP.register(m, :rosenbrock, 2, rosenbrock, autodiff=true)
```
| argument | description |
| ------ | -------- |
| `m` | model |
| `:rosenbrock` | Symbol used to identify funciton in objective|
| `2` | number of scalar inputs |
| `rosenbrock` | function we've declared |

If you can't forward diff your package, or you have an optimized gradient, you can also pass those in explicitly.  Note that JuMP currently doesn't support Hessians for functions of more than one variable.

In [31]:
f(x,y) = x^2 + y^2
function ∇f(g,x,y) 
    g[1] = 2*x
    g[2] = 2*y
end

using JuMP, Ipopt

m = Model(solver=IpoptSolver())

# registers function with JuMP, and derivatives are computed
JuMP.register(m, :f, 2, f, ∇f)

@variable(m, x, start = 1.0)
@variable(m, y, start = 1.0)

@NLobjective(m, Min, f(x,y))

solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))


This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  